In [ ]:
!pip install gensim


In [44]:
import gensim

import gensim.downloader

Part 1: Word Embeddings

In [ ]:
# Load pre-trained Word2Vec model trained on wikipedia data

model = gensim.downloader.load("glove-wiki-gigaword-100")
# Getting the vector for a word

dog = model['dog']

# Getting the most similar words

print(model.most_similar('dog'))

# Getting the similarity between two words

print(model.similarity('dog', 'cat'))


Part 1a

In [46]:

#find the most similar words to a word of your choice. 
#Does the result make sense? Look up some of the words if you don't know them
#Share and discuss with your group

In [ ]:
papaya = model['papaya']
print(model.most_similar('papaya'))

In [ ]:
juice = model['juice']
print(model.most_similar('juice'))

In [ ]:
kpop = model['kpop']
print(model.most_similar('kpop'))

In [ ]:
thailand = model['thailand']
print(model.most_similar('thailand'))

In [ ]:

#we can extract a "capital vector" by subtracting the vector for a country from the vector for its capital
capital= model["berlin"]-model['germany']

fra_capital=model["france"]+capital

print(model.most_similar(fra_capital))
#likewise, we can extract a "plural vector" by subtracting the vector for a singular noun from the vector for its plural
plural=model["cats"]-model["cat"]

print(model.most_similar(plural))

#we can also add vectors together
plural_dogs=model["dog"]+plural

print(model.most_similar(plural_dogs))



In [ ]:
# this can be used to find examples of data bias in word embeddings

# for example, we can find the most similar words to "doctor" and "nurse"

gender=model["woman"]-model["man"]

print(model.most_similar(gender))
print(model.most_similar(-gender))

model.most_similar(model["doctor"]+gender)



Part 1b

In [53]:
#try to isolate a specific dimension in the embeddings and add it to multiple words. 
#does this show any bias or assumptions in the data? 
# Discuss with your group and share most interesting dimension in the document

In [ ]:
plural=model["cats"]-model["cat"]
plural_papayas = model["papaya"]+plural
print(model.most_similar(plural_papayas))

In [ ]:
plural_juices = model["juice"]+plural
print(model.most_similar(plural_juices))

In [ ]:
plural_kpops = model["kpop"]+plural
print(model.most_similar(plural_kpops))

In [ ]:
plural_thailands = model["thailand"]+plural
print(model.most_similar(plural_thailands))

Part 2: Visualization and Embeddings in Space

In [ ]:
#visualize pca of word embeddings

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

words=["dog","cat","fish", "whale", "bird", "eagle", "lion", "tiger", "cheetah", "elephant", "giraffe", "zebra", "kangaroo", "koala", "panda", "monkey", "gorilla", "chimpanzee", "orangutan", ]

X=[model[word] for word in words]

#identify clusters of words in the word embeddings at high dimensions

pca=PCA(n_components=2)

X_pca=pca.fit_transform(X)

plt.scatter(X_pca[:,0],X_pca[:,1])

for i, word in enumerate(words):
    plt.annotate(word, (X_pca[i,0],X_pca[i,1]))

plt.show()

In [ ]:
#repeat this process with your own set of 10-20 words. 
#Does the proximity between words and overall distances make sense? Work as a group and discuss your individual findings.
words=["banana", "mountain", "robot", "beach", "galaxy", "engineer", "happiness", "piano", "airplane", "library", "desert", "philosophy", "whale", "museum", "molecule"]

X=[model[word] for word in words]

#identify clusters of words in the word embeddings at high dimensions

pca=PCA(n_components=2)

X_pca=pca.fit_transform(X)

plt.scatter(X_pca[:,0],X_pca[:,1])

for i, word in enumerate(words):
    plt.annotate(word, (X_pca[i,0],X_pca[i,1]))

plt.show()

In [ ]:
# repeat the process with a different pre-trained model and the same set of words. 
# What has changed in the embedding space? What might this have to do with the training data?
twitter_model= gensim.downloader.load("glove-twitter-100")
words=["banana", "mountain", "robot", "beach", "galaxy", "engineer", "happiness", "piano", "airplane", "library", "desert", "philosophy", "whale", "museum", "molecule"]

X=[twitter_model[word] for word in words]

#identify clusters of words in the word embeddings at high dimensions

pca=PCA(n_components=2)

X_pca=pca.fit_transform(X)

plt.scatter(X_pca[:,0],X_pca[:,1])

for i, word in enumerate(words):
    plt.annotate(word, (X_pca[i,0],X_pca[i,1]))

plt.show()


Part 3: Embedding reddit posts

In [61]:
#read in uiuc csv
import pandas as pd

uiuc=pd.read_csv("uiuc.csv")
mich=pd.read_csv("umich.csv")

#sample so we have even number of samples from each dataset
mich=mich.sample(n=4725)

#assign labels based on origin subreddit of comment
uiuc['label']=1
mich['label']=0

#you will be working with the data csv for the rest of the question
data=pd.concat([uiuc,mich])


In [62]:
#using the code from project 1, clean the text and create a bag of words representation of the text
def clean(text):
    text = ' '.join(text.split())
    text = text.lower()
    text = ''.join([char for char in text if char.isalpha() or char.isspace()])
    return text

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
data['cleaned_text'] = data['text'].apply(clean)
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(data['cleaned_text'])

In [ ]:
bag_of_words

In [65]:
#using the embedding code from last week, plot a pca of all posts in the subreddits data set

#color by the "label" column to see if the two subreddits are separable in the embedding space

# is there a clear separation between the two columns? 

In [ ]:
sample_size = 2000
bag_of_words_sampled = bag_of_words[:sample_size].toarray()
pca = PCA(n_components=2)
pca_result = pca.fit_transform(bag_of_words_sampled)
plt.figure(figsize=(10, 7))
plt.scatter(pca_result[:, 0], pca_result[:, 1], c=data['label'][:sample_size], cmap="coolwarm", alpha=0.5)
plt.colorbar(label="Label")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("PCA of Sampled UIUC and UMich Subreddits")
plt.show()

In [ ]:
#here is a function to calculate the average embedding of a cleaned document
import numpy as np
#we can sum together all the embeddings to get a representation of all the concepts expressed in the document
def avg_embedding(sentence, model):
    words=sentence.split()
    embeddings=[model[word] for word in words if word in model]
    return np.mean(embeddings, axis=0)

sentences=["i am running today", "i will jog tomorrow", "i ran yesterday", "i love drawing", "i adore painting", "i used to like pottery"]

#we can then calculate the average embedding of each sentence

avg_embeddings=[avg_embedding(sentence, model) for sentence in sentences]

#we can then plot the pca of these embeddings

pca=PCA(n_components=2)

X_pca=pca.fit_transform(avg_embeddings)

plt.scatter(X_pca[:,0],X_pca[:,1])

for i, sentence in enumerate(sentences):
    plt.annotate(sentence, (X_pca[i,0],X_pca[i,1]))

plt.show()

In [68]:

#use this function to calculate the average embedding of each post in the dataset

#plot the pca of the average embeddings, color by the "label" column

#how is this plot different from the previous one?

#is there clear separation? 

In [ ]:
#here is a function to calculate the average embedding of a cleaned document

data['avg_embedding'] = data['cleaned_text'].apply(lambda x: avg_embedding(x, model))
avg_embeddings=[avg_embedding(sentence, model) for sentence in data['cleaned_text']]
avg_embeddings=[embedding for embedding in avg_embeddings if not np.isnan(embedding).any()]

pca=PCA(n_components=2)

X_pca=pca.fit_transform(avg_embeddings)

plt.scatter(X_pca[:,0],X_pca[:,1])

for i, sentence in enumerate(sentences):
    plt.annotate(sentence, (X_pca[i,0],X_pca[i,1]))

plt.show()

Part 4: HuggingFace and BERT

In [ ]:
!pip install transformers

In [ ]:
#Please wait until after next week to start

# Use a pipeline as a high-level helper
from transformers import pipeline

#download bert model
pipe = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis")


In [ ]:
#use the pipeline to classify the sentiment of a sentence of your choice
pipe("I love this class")

In [ ]:
#use the pipeline to classify the sentiment of all posts in the reddit dataset
#report the most common sentiment label
def classify_sentiment_batch(texts, batch_size=32):
    sentiments = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_texts = [text[:512] if text.strip() else "neutral" for text in batch_texts]
        try:
            batch_results = pipe(batch_texts)
            batch_labels = [result['label'] for result in batch_results]
            sentiments.extend(batch_labels)
        except Exception as e:
            print(f"Error in batch {i // batch_size}: {e}")
            sentiments.extend(["ERROR"] * len(batch_texts))
    return sentiments

data['sentiment'] = classify_sentiment_batch(data['cleaned_text'].tolist())
most_common_sentiment = data['sentiment'].mode()[0]
print(f"The most common sentiment label is: {most_common_sentiment}")

In [ ]:
#look at ten sample posts and labels; do you agree with all of them? 
sample_posts = data[['text', 'sentiment']].sample(n=10)
for index, row in sample_posts.iterrows():
    print(f"Post: {row['text']}")
    print(f"Predicted Sentiment: {row['sentiment']}")
    print("\n")

In [ ]:
#is the UIUC or Michigan subreddit more negative?
uiuc_negative = data[(data['label'] == 1) & (data['sentiment'] == 'NEG')].shape[0]
mich_negative = data[(data['label'] == 0) & (data['sentiment'] == 'NEG')].shape[0]

uiuc_total = data[data['label'] == 1].shape[0]
mich_total = data[data['label'] == 0].shape[0]

uiuc_negativity = uiuc_negative / uiuc_total
mich_negativity = mich_negative / mich_total 

uiuc_negativity
mich_negativity